In [1]:
#basic package
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import time
import datetime as dt
import itertools
import re
import math
from collections import Counter
import operator
from dtaidistance import dtw
import random
from sklearn.preprocessing import normalize
import pickle #to save/load list of selected hens

#plot
import matplotlib.pyplot as plt
import seaborn as sns

#network
import networkx as nx

In [2]:
PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
from UTILS import chi2_distance, correct_key
import config_origins as config

C:\Users\camil\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


2


# Define parameters

In [3]:
print('change the configuration file if not done yet!')
path_extracted_data = config.path_extracted_data
path_initial_data = config.path_initial_data
id_run = config.id_run
#id_run = 'chapter0_final_'
#path_dataoutput = r'G:\VPHI\Welfare\2- Research Projects\OFHE2.OriginsE2\DataOutput'
#path_extracted_data = os.path.join(path_dataoutput,'TrackingSystem') 
#path_extracted_data = os.path.join(path_extracted_data, id_run)
dico_night_hour = config.dico_night_hour
dico_matching = config.dico_matching
li_binmn = config.li_binmn
penalty = config.penalty
dico_window = config.dico_window
path_extracted_data_SNA = config.path_extracted_data_SNA
#create a director if not existing
if not os.path.exists(path_extracted_data_SNA):
    os.makedirs(path_extracted_data_SNA)
print(id_run)

change the configuration file if not done yet!
correctlightschedule_


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Download variables

#focal birds info (one row per bird)
df_FB = pd.read_csv(os.path.join(path_extracted_data,id_run+'df_FOCALBIRDS.csv'), sep=';', parse_dates=['InitialStartDate'],
                     dayfirst=True) 
df_FB['percentage_of_gain_weight'] = df_FB.apply(lambda x: (x['weight 23-11-2020']-x['29-09 weight'])/x['29-09 weight']*100, axis=1)
print(df_FB.shape)
df_FB.head(3)

#cleaned data of the tracking system movements
df = pd.read_csv(os.path.join(path_extracted_data, id_run+'_CLEANEDDATA.csv'), sep=';', 
                 parse_dates=['Timestamp', 'date'], dayfirst=True) 
df['hour'] = df['Timestamp'].map(lambda x: x.hour)
df['time'] = df['Timestamp'].map(lambda x: dt.datetime.time(x-dt.timedelta(seconds=x.second)))
df.drop('duration', axis=1, inplace=True)
print(df.shape)
df.head(3)

In [4]:
#daily variables (one row per (henID, date))
#daily_ALL_Variable_Tranformed
df_daily = pd.read_csv(os.path.join(path_extracted_data, id_run+'_daily_ALL_variables.csv'), sep=';',
                     parse_dates=['level','FirstTimestamp_1_Zone', 'FirstTimestamp_2_Zone', 'FirstTimestamp_3_Zone',
                              'FirstTimestamp_4_Zone', 'FirstTimestamp_5_Zone', 
                              'Nestbox_time_of_first_staid_longer_than900sec',
                              'duration_last-firsttransition_mn'], dayfirst=True) 
df_daily['DOA'] = df_daily['level'].map(lambda x: (x-dt.datetime(2020,6,3)).days) 
df_daily['weeks_in_laying_barn'] = df_daily['DOA'].map(lambda x: int((x-119)/7)+1)
#df_daily['weeks_in_laying_barn'] = df_daily['DOA'].map(lambda x: math.ceil(x/7))
#first two month seems good from the %of hens not moving plot! and sounds good too (twice longer thatn they need to start moving)
print(df_daily.shape)

#filter by date
df_daily = df_daily[df_daily['level']>dt.datetime(2020,9,29)]

#remove days that are not fully recorded
df_daily['nbr_sec_per_day'] = df_daily['level'].map(lambda x: dico_night_hour[correct_key(x,dico_night_hour)]['nbr_hour']*60*60)
df_daily['is_correct_amount_time'] = df_daily.apply(lambda x: x['nbr_sec_per_day']==x['verification_daily_total_duration'], axis=1)
df_daily[(~df_daily['is_correct_amount_time'])&(~df_daily['Total_number_transition'].isnull())][['level', 'HenID', 'Total_number_transition', 'dur_values', 'verification_daily_total_duration','nbr_sec_per_day']]
print(df_daily.shape)
display(df_daily = df_daily[~((~df_daily['is_correct_amount_time'])&(~df_daily['Total_number_transition'].isnull()))])
print(df_daily.shape)

#remove the days where the night had movement recorded but not the days
print(df_daily.shape)
#display(df_daily[df_daily.isna().any(axis=1)])
df_daily = df_daily[~df_daily['verification_daily_total_duration'].isnull()]
print(df_daily.shape)

df_daily['dur_values_normalized'].replace('[nan, nan, nan, nan, nan]','[np.nan,np.nan,np.nan,np.nan,np.nan]', inplace=True)
df_daily['dur_values_normalized'] = df_daily['dur_values_normalized'].map(lambda x: eval(x))
df_daily['duration_last-firsttransition_mn'] = df_daily['duration_last-firsttransition_mn'].astype(float)
print(df_daily.shape)
df_daily.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\camil\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (103) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(42515, 149)
(42370, 151)
(42370, 151)
(42370, 151)
(32032, 151)
(32032, 151)


,HenID,level,duration_1_Zone,duration_2_Zone,duration_3_Zone,duration_4_Zone,duration_5_Zone,verification_daily_total_duration,dur_values,dur_values_normalized,...,temperature_night20_2_var,list_of_MovementCounter_day,len_MovementCounter_day,MovementCounter_day_amount_nnactivity,MovementCounter_day_max,MovementCounter_day_mean,DOA,weeks_in_laying_barn,nbr_sec_per_day,is_correct_amount_time
0,hen_130,2020-10-04,0.0,140.0,1166.0,79.0,30613.0,31998.0,"[0.0, 140.0, 1166.0, 79.0, 30613.0]","[0.0, 0.004375273454590912, 0.0364397774860928...",...,NaN,NaN,NaN,NaN,NaN,NaN,123,1,32400,False
1,hen_130,2020-10-05,0.0,0.0,0.0,0.0,32400.0,32400.0,"[0.0, 0.0, 0.0, 0.0, 32400.0]","[0.0, 0.0, 0.0, 0.0, 1.0]",...,NaN,NaN,NaN,NaN,NaN,NaN,124,1,32400,True
2,hen_130,2020-10-06,0.0,0.0,780.0,325.0,31295.0,32400.0,"[0.0, 0.0, 780.0, 325.0, 31295.0]","[0.0, 0.0, 0.024074074074074074, 0.01003086419...",...,NaN,NaN,NaN,NaN,NaN,NaN,125,1,32400,True


# Distance across pairs of MLPS from same day - for daily SNA

In [5]:
#download the MLPs vectors during day only! as we compare on same day, we can restrict to the exact daily calendar!
dico_pen_bin_zone_level_h = pickle.load(open(os.path.join(path_extracted_data, 
                                                 id_run+'dico_pen_bin_zone_level_h_DAILYLEVEL.pkl'), 'rb'))
dico_pen_bin_level_h = pickle.load(open(os.path.join(path_extracted_data, 
                                                     id_run+'dico_pen_bin_level_h_DAILYLEVEL.pkl'), 'rb'))
#dico of hen as keys and pen as values
df_daily['PenID'] = df_daily['PenID'].map(lambda x: 'pen'+(str(int(x))))
dico_h_p = dict(zip(df_daily['HenID'].tolist(), df_daily['PenID'].tolist()))

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
#TODO pour DTW on devrais avoir un autre vecteur!! avec des 1-0 seulement
dico_pen_bin_zone_level_h['pen8'][30]['5_Zone'][dt.datetime(2020,9,29)]['hen_126']

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 11.7,
 30.0,
 30.0,
 30.0,
 30.0,
 30.0,
 30.0,
 30.0,
 30.0,
 7.783333333333333]

In [ ]:
li_binmn = [30]

In [ ]:
#compute the CHI2DISTANCE & DTW for each bird with all other birds
#start recording the time it last
START_TIME = time.perf_counter()
li_df = []
#df.sort_values(['Timestamp'], inplace=True)
#take (subset of) dates and hens that are available (i.e. for which we comptue the variable on)
#df_daily_h = df_daily[(df_daily['DOA']>=150)&(df_daily['DOA']<=380)].copy()
li_date = set(df_daily['level'].tolist())
li_zone = ['1_Zone', '2_Zone', '3_Zone', '4_Zone', '5_Zone']
for d in tqdm.tqdm(sorted(list(li_date))):
    li_hen_d = df_daily[df_daily['level']==d]['HenID'].unique()
    #we use symmetric measures, so we only do each combination once
    for i,h1 in enumerate(li_hen_d):

        #lets compute for hen h1 its difference with all other birds
        for h2 in li_hen_d[i+1:]:
            
            try:
                ############ Overall aviary usage similarity across days
                l1_chi2 = df_daily[(df_daily['HenID']==h1)&(df_daily['level']==d)]['dur_values_normalized'].values[0]
                l2_chi2 = df_daily[(df_daily['HenID']==h2)&(df_daily['level']==d)]['dur_values_normalized'].values[0]
                dico_ = {'date':d, 'henId':h1, 'henID2':h2, 'chi2distance':chi2_distance(l1_chi2,l2_chi2, remove_warning=True)}
                #'dur_values_normalized_henId':l1_chi2,'dur_values_normalized_henID2':l2_chi2,

                ############# Specifics' zones usage similarity across days
                for nbr_binmn in li_binmn:
                    #ALL zone together with chi2distance
                    l1_chi2 = dico_pen_bin_level_h[dico_h_p[h1]][nbr_binmn][d][h1]
                    l2_chi2 = dico_pen_bin_level_h[dico_h_p[h2]][nbr_binmn][d][h2]
                    dico_['nbr_obs_chi2all_'+str(nbr_binmn)] = len(l1_chi2)
                    li_chi2 = [chi2_distance(l1_chi2[i],l2_chi2[i], remove_warning=True) for i in range(0,len(l1_chi2))]
                    dico_['li_chi2_'+str(nbr_binmn)] = li_chi2
                    dico_['chi2distance_ALL_'+str(nbr_binmn)] = np.mean(li_chi2)

                    #per zone with DTW
                    for ZONE in li_zone:
                        dtw_value = np.nan
                        try:
                            l1_dtw = np.array(dico_pen_bin_zone_level_h[dico_h_p[h1]][nbr_binmn][ZONE][d][h1], dtype=np.double)
                            l2_dtw = np.array(dico_pen_bin_zone_level_h[dico_h_p[h2]][nbr_binmn][ZONE][d][h2], dtype=np.double)
                            #compute distance measure
                            #psi: #of start and end points of a sequence can be ignored if this would lead to a lower distance
                            dtw_value, paths = dtw.warping_paths(l1_dtw, l2_dtw, window=dico_window[nbr_binmn], psi=0, penalty=penalty)
                        except Exception as e:
                            print(e)
                            sys.exit()
                            pass #dont do anything
                        dico_['DTW_'+str(nbr_binmn)+'_'+ZONE] = round(dtw_value,1)  
                        dico_['nbr_obs_l1_'+str(nbr_binmn)+'_'+ZONE] = len(l1_dtw)
                        dico_['nbr_obs_l2_'+str(nbr_binmn)+'_'+ZONE] = len(l2_dtw)
                li_df.append(dico_)
            except Exception as e:
                print(e)
                print(h1, h2)
                print(d)
                
END_TIME = time.perf_counter()
print ("Total running time: %.2f mn" %((END_TIME-START_TIME)/60))
#turn inter and intra distances into a dataframe                
df_inter = pd.DataFrame(li_df)
#df_inter = df_inter[~df_inter['chi2distance'].isnull()]
df_inter['hen_pair'] = df_inter.apply(lambda x: '-'.join(sorted([x['henId'],x['henID2']])), axis=1)
df_inter['DTW_30_all'] = df_inter[['DTW_30_1_Zone', 'DTW_30_2_Zone', 'DTW_30_3_Zone','DTW_30_4_Zone','DTW_30_5_Zone']].sum(axis=1)
li_col = list(df_inter.columns)
li_remove = [i for i in li_col if i.startswith('li_chi2_')]
print(li_remove)
li_keep = [i for i in li_col if i not in li_remove]
#df_inter.iloc[0:1000].to_csv(os.path.join(path_extracted_data_visual_adap, id_run+'_df_DistanceBetweenHenSim_4verification.csv'), sep=';', index=False)
#df_inter.filter(li_keep).to_csv(os.path.join(path_extracted_data_visual_adap ,id_run+'_df_DistanceBetweenHenSim.csv'), sep=';', index=False)
df_inter.to_csv(os.path.join(path_extracted_data ,id_run+'_df_DistanceBetweenHenSim_DAILYLEVEL.csv'), sep=';', index=False)
print(df_inter.shape)
display(df_inter.head(3))

In [ ]:
print(df_inter.shape)
display(df_inter.head(3))

# Distance across any pairs of MLPS - for daily clusterID

In [ ]:
#download MLPs vectors
#For now we restrict to 2h-17h in order to always compare same length TS as we will compare across days, the lenght might differ
dico_pen_bin_level_h = pickle.load(open(os.path.join(path_extracted_data_visual_adap, 
                                                     id_run+'dico_pen_bin_level_h_2h-17h59LEVEL.pkl'), 'rb'))

In [ ]:
#compute mvt distances between any two MLPs on any day from any pen
dico_hendate_mvt = {}
for p,dico_bin_level_h in dico_pen_bin_level_h.items():
    for level, dico_hen_mvt in dico_bin_level_h[15].items():
        if level!=dt.datetime(2020,9,29):
            for henID,limvt in dico_hen_mvt.items():
                dico_hendate_mvt[henID+'/'+str(level).split(' ')[0]] = limvt
dico_hendate_mvt = OrderedDict(dico_hendate_mvt)
li_hendate = list(dico_hendate_mvt.keys())
li_mvt = list(dico_hendate_mvt.values())
len(dico_hendate_mvt)

In [ ]:
li_df = []
for i in tqdm.tqdm(range(0,len(li_mvt))):
    li_mvt1 = li_mvt[i]
    for j in range(i+1,len(li_mvt)):
        dico_ = {}
        li_mvt2 = li_mvt[j]
        li_chi2 = [chi2_distance(li_mvt1[i],li_mvt2[i]) for i in range(0,len(li_mvt1))]
        dico_['chi2distance_ALL_15'] = np.mean(li_chi2)
        dico_['h1'] = li_hendate[i]
        dico_['h2'] = li_hendate[j]
        li_df.append(dico_)

In [ ]:
df_anypair = pd.DataFrame(li_df)
display(df_anypair[df_anypair['chi2distance_ALL_15'].isnull()])
df_anypair['henID1'] = df_anypair['h1'].map(lambda x: x.split('/')[0])
df_anypair['henID2'] = df_anypair['h2'].map(lambda x: x.split('/')[0])
df_anypair['level1'] = df_anypair['h1'].map(lambda x: dt.datetime.strptime(x.split('/')[1], '%Y-%m-%d'))
df_anypair['level2'] = df_anypair['h2'].map(lambda x: dt.datetime.strptime(x.split('/')[1], '%Y-%m-%d'))
#df_interintra['weeks_in_laying_barn'] = df_interintra['DOA'].map(lambda x: int((x-119)/7)+1)
#df_interintra['hen_pair'] = df_interintra.apply(lambda x: '-'.join(sorted([x['henID1'],x['henID2']])), axis=1)
df_anypair.to_csv(os.path.join(path_extracted_data_visual_adap ,id_run+'_df_DistanceBetweenAnyDaysHenSim_2h-17h59LEVEL.csv'), sep=';', index=False)
print(df_anypair.shape)
display(df_anypair.head(3))

In [ ]:
plt.hist(df_anypair['chi2distance_ALL_15']);

In [31]:
l1 = np.array(['ad','ad','er','ww'])
l2 = np.array(['ad','ed','er','ww'])
sum(l1==l2)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


3